# Análise do Cenário de Retorno para Pouso no Brasil

## Introdução
Falhas podem ocorrer em qualquer momento depois da decolagem. 

Esta página permite uma visualização da exposição a este risco na aviação regular brasileira.

Em resumo: **quero saber de que aeroportos com pista curta existe operação regular de aeronaves sem que exista um aeroporto de pista mais longa próximo**.

## Como Usar
1. Clique no menu Kernel.
2. Clique em "Restart & Run All".
3. Clique no botão vermelho da caixa que vai se abrir: "Restart and Run All Cells".
4. Ignore o código (ou não :D ) e vá até o final da página onde aparece um mapa.
5. Experimente mudar as opções nas duas caixas e na barra de seleção disponíveis. 
6. O mapa deve se atualizar a cada modificação das seleções.
7. A explicação do mapa está abaixo dele.

In [1]:
import pandas as pd
import ipywidgets as widgets
import folium
from ipywidgets import interact, widgets

In [2]:
world_rwy = pd.read_csv('data/world_rwy.csv')
database = pd.read_csv('data/database.csv')

In [3]:
def create_map(equipamento, comp_min = '8000', dist = 100):
    m = folium.Map(location = [-15.8267,-55.9218], zoom_start = 4, width = 600, height = 600)

    for x in world_rwy[world_rwy['COMPRIMENTO_MAX'] > float(comp_min)][['LATITUDE','LONGITUDE']].values:
        folium.vector_layers.Circle(tuple(x[0:2]), 1852*dist, stroke = False, fill_color = "green").add_to(m)


    for x in database[(database['Dist_' + comp_min] > dist) & (database['Type'] == equipamento)][['LATITUDE','LONGITUDE','Cód Origem']].values:
        folium.Marker(location=tuple(x[0:2]), tooltip = x[2], icon=folium.Icon(color='red')).add_to(m)

    for x in database[((database['Dist_' + comp_min] <= dist) & (database['Dist_' + comp_min] > 0.0)) & (database['Type'] == equipamento)][['LATITUDE','LONGITUDE','Cód Origem']].values:
        folium.Marker(location=tuple(x[0:2]), tooltip = x[2], icon=folium.Icon(color='blue')).add_to(m)

    return m

In [6]:
# interact(create_map(equip), 
#          equip = widgets.Dropdown(
#                     options=['Turboprop', 'Regional Jet', 'Narrow Body', 'Wide Body'],
#                     value='Narrow Body',
#                     description='Categoria de Avião:',
#                     disabled=False,
#                 ))
         

@interact
def cria_mapa(equipamento = widgets.Dropdown(
                                 options=['Turboprop', 'Regional Jet', 'Narrow Body', 'Wide Body'],
                                 value='Narrow Body',
                                 description='Categoria:',
                                 disabled=False),
               comprimento_pista_ft = widgets.Dropdown(
                                 options=['4000','6000','8000','10000','12000'],
                                 value='8000',
                                 description='Pista [ft]',
                                 disabled=False),
                  dist_nm  = widgets.IntSlider(
                                        value=200,
                                        min=0,
                                        max=400,
                                        step=50,
                                        description='Dist. [nm]:',
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='d')):
    return create_map(equipamento, comprimento_pista_ft,dist_nm)

interactive(children=(Dropdown(description='Categoria:', index=2, options=('Turboprop', 'Regional Jet', 'Narro…

## O que estou vendo no mapa?
As duas caixas de seleção permitem respectivamente selecionar:
* a classificação das aeronaves;
* o comprimento mínimo de pista a ser considerado.

Os **círculos verdes** marcam a **distância selecionada na barra de seleção a partir de um aeroporto que tem pelo menos uma pista com comprimento maior do que o selecionado**.

Os **marcadores** no mapa mostram **aeroportos com voos regulares deste tipo de aeronave cuja pista de decolagem é inferior ao valor selecionado**. Eles são pintados:
* **azul** - se existe uma pista com comprimento mairo do que o escolhido dentro da distância escolhida
* **vermelho** - se **não** existe uma pista com comprimento maior do que o escolhido dentro da distância escolhida 





## Como a página funciona

Em resumo ela apenas plota resultados de dois databases construídos pelo autor com uma combinação dos seguintes databases públicos:

* [ANAC - Cadastro de Aeródromos Privados](https://www.anac.gov.br/dadosabertos/areas-de-atuacao/aerodromos/aerodromos-cadastro-de-aerodromos-privados-csv-1)
* [ANAC - Cadastro de Aeródromos Públicos](https://www.anac.gov.br/dadosabertos/areas-de-atuacao/aerodromos/aerodromos-cadastro-de-aerodromos-publicos-csv)
* [ANAC - Lista de todos os serviços de transporte aéreo público registrados Vigentes](https://sistemas.anac.gov.br/sas/registros/Diario/diario.csv)
* [Our Airports - Airports](http://ourairports.com/data/airports.csv)
* [Our Airports - Runways](http://ourairports.com/data/runways.csv)